In [1]:
import glob
import os
import numpy as np
import tensorflow as tf
from music21 import *
import random
from matplotlib import pyplot as plt
from fractions import Fraction
import json

NUM_PIANO_KEYS = 88
A0_MIDI_OFFSET = 21

In [2]:
notes = []
offsets = []
durations = []
#glob.glob('midi_data/scales/**/*.mid', recursive=True)
#filenames = glob.glob('maestro-v3.0.0/**/*.mid*', recursive=True)
#maestro-v3.0.0\\2008\MIDI-Unprocessed_09_R1_2008_01-05_ORIG_MID--AUDIO_09_R1_2008_wav--3.midi
#G_major.mid
for file in glob.glob('jazz_solos_midi/Swing/*.mid', recursive=True):
    mid = converter.parse(file)
    notes_to_parse = None
    prev_offset = 0
          
    notes_to_parse = mid.flatten().notes
    
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            arr = np.zeros(NUM_PIANO_KEYS)
            try:
                arr[element.pitch.midi - A0_MIDI_OFFSET] = 1
            except IndexError:
                os.remove(file)
                break
                
        
            notes.append(arr)
            
            durations.append(str(element.quarterLength))
            
            offsets.append(round(float(element.offset - prev_offset), 3))
            prev_offset = element.offset
            
        elif isinstance(element, chord.Chord):
            isFirstNote = True
            for n in element:
                arr = np.zeros(NUM_PIANO_KEYS)
                try:
                    arr[n.pitch.midi - A0_MIDI_OFFSET] = 1
                except IndexError:
                    try:
                        os.remove(file)
                        break
                    except FileNotFoundError:
                        break
                    
                notes.append(arr)
                
                durations.append(str(n.quarterLength))
                
                if isFirstNote:
                    offsets.append(round(float(element.offset - prev_offset), 3))
                    prev_offset = element.offset
                    isFirstNote = False
                else:
                    offsets.append(float(0))

notes = np.asarray(notes)
len(notes)

2472

In [3]:
# Used to read in previous dictionaries, used for when transfer learning
# so that the encodings were consistent between models
offset_map = dict()
reverse_duration = dict()
with open('mappings/offset_map.txt') as f:
    for line in f:
        key, value = line.strip().split(":")
        offset_map[float(key)] = int(value)
with open('mappings/duration_map.txt') as json_file:
    duration_map = json.load(json_file)  
with open('mappings/reverse_duration.txt') as f:
    for line in f:
        key, value = line.strip().split(":")
        reverse_duration[int(key)] = value
reverse_offset = dict()
with open('mappings/reverse_offset.txt') as f:
    for line in f:
        key, value = line.strip().split(":")
        reverse_offset[int(key)] = float(value)

In [4]:
duration_map

{'0.0': 0,
 '0.25': 1,
 '0.5': 2,
 '0.75': 3,
 '1.0': 4,
 '1.25': 5,
 '1.5': 6,
 '1.75': 7,
 '1/12': 8,
 '1/3': 9,
 '1/6': 10,
 '10/3': 11,
 '11/12': 12,
 '11/3': 13,
 '11/6': 14,
 '13/12': 15,
 '13/6': 16,
 '17/12': 17,
 '17/6': 18,
 '19/12': 19,
 '19/6': 20,
 '2.0': 21,
 '2.25': 22,
 '2.5': 23,
 '2.75': 24,
 '2/3': 25,
 '23/12': 26,
 '23/6': 27,
 '25/12': 28,
 '29/12': 29,
 '3.0': 30,
 '3.25': 31,
 '3.5': 32,
 '3.75': 33,
 '31/12': 34,
 '35/12': 35,
 '37/12': 36,
 '4.0': 37,
 '4.25': 38,
 '4/3': 39,
 '41/12': 40,
 '43/12': 41,
 '47/12': 42,
 '5/12': 43,
 '5/3': 44,
 '5/6': 45,
 '7/12': 46,
 '7/3': 47,
 '7/6': 48,
 '8/3': 49}

In [5]:
offset_map

{0.0: 0,
 0.083: 1,
 0.167: 2,
 0.25: 3,
 0.333: 4,
 0.417: 5,
 0.5: 6,
 0.583: 7,
 0.667: 8,
 0.75: 9,
 0.833: 10,
 0.917: 11,
 1.0: 12,
 1.083: 13,
 1.167: 14,
 1.25: 15,
 1.333: 16,
 1.417: 17,
 1.5: 18,
 1.583: 19,
 1.667: 20,
 1.75: 21,
 1.833: 22,
 1.917: 23,
 2.0: 24,
 2.083: 25,
 2.167: 26,
 2.25: 27,
 2.333: 28,
 2.417: 29,
 2.5: 30,
 2.583: 31,
 2.667: 32,
 2.75: 33,
 2.833: 34,
 2.917: 35,
 3.0: 36,
 3.083: 37,
 3.167: 38,
 3.25: 39,
 3.333: 40,
 3.417: 41,
 3.5: 42,
 3.583: 43,
 3.667: 44,
 3.75: 45,
 3.833: 46,
 3.917: 47,
 4.0: 48,
 4.083: 49,
 4.167: 50,
 4.25: 51,
 4.333: 52,
 4.417: 53,
 4.5: 54,
 4.583: 55,
 4.667: 56,
 4.75: 57,
 4.833: 58,
 4.917: 59,
 5.0: 60,
 5.083: 61,
 5.167: 62,
 5.25: 63,
 5.333: 64,
 5.417: 65,
 5.5: 66,
 5.583: 67,
 5.667: 68,
 5.75: 69,
 5.833: 70,
 5.917: 71,
 6.0: 72,
 6.083: 73,
 6.167: 74,
 6.25: 75,
 6.333: 76,
 6.417: 77,
 6.5: 78,
 6.583: 79,
 6.667: 80,
 6.75: 81,
 6.833: 82,
 6.917: 83,
 7.0: 84,
 7.083: 85,
 7.167: 86,
 7.25: 87,

In [6]:
temp = []
size = len(duration_map)
for duration in durations:
    arr = np.zeros(size)
    arr[duration_map[duration]] = 1
    temp.append(arr)
durations = np.asarray(temp)

In [7]:
temp = []
size = len(offset_map)
for offset in offsets:
    arr = np.zeros(size)
    try:
        arr[offset_map[offset]] = 1
    except IndexError:
        print(offset, offset_map[offset])
        print(arr[offset_map[offset]])
    temp.append(arr)
offsets = np.asarray(temp)

In [8]:
train_notes = np.concatenate([notes, durations, offsets], axis = 1)

seq_length = 50
inputs = []
note_outputs = []
duration_outputs = []
offset_outputs = []
for i in range(len(notes) - seq_length):
    inputs.append(train_notes[i : i + seq_length])
    
    note_outputs.append(notes[i + seq_length])
    duration_outputs.append(durations[i + seq_length])
    offset_outputs.append(offsets[i + seq_length])
    
input_size = len(inputs)
inputs = np.asarray(inputs)
inputs.reshape(input_size, seq_length, len(train_notes[0]))

note_outputs = np.asarray(note_outputs)
duration_outputs = np.asarray(duration_outputs)
offset_outputs = np.asarray(offset_outputs)

In [9]:
input_shape = inputs[0].shape
learning_rate = 0.005

In [10]:
inp = tf.keras.Input(input_shape)
lstm = tf.keras.layers.LSTM(512)(inp)
drop = tf.keras.layers.Dropout(0.25)(lstm)
dense = tf.keras.layers.Dense(128)(drop)
out = {
    "pitch": tf.keras.layers.Dense(NUM_PIANO_KEYS, name = "pitch", activation = "softmax")(dense),
    "duration": tf.keras.layers.Dense(len(duration_map), name = "duration", activation = "softmax")(dense),
    "offset": tf.keras.layers.Dense(len(offset_map), name = "offset", activation = "softmax")(dense),
}

In [11]:
model2 = tf.keras.Model(inp, out)
model2.compile(
    loss = "categorical_crossentropy",
    loss_weights = {
        'pitch': 1.0,
        'step': 1.0,
        'duration':1.0,
    },
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
    metrics = ["accuracy"]
)
model2.load_weights("weights/jazznet")
model2.weights

[<tf.Variable 'lstm/lstm_cell/kernel:0' shape=(373, 2048) dtype=float32, numpy=
 array([[-0.02549937, -0.02157745,  0.04676175, ..., -0.00208347,
          0.0109541 ,  0.03197775],
        [ 0.0097643 , -0.02284117,  0.00407151, ...,  0.03663059,
         -0.00645186,  0.03012   ],
        [-0.02757075, -0.02471986, -0.00685724, ..., -0.04264772,
          0.03959554, -0.04600886],
        ...,
        [-0.0039024 ,  0.00626381, -0.00258368, ...,  0.02822074,
          0.01161296,  0.01594645],
        [ 0.00987686,  0.04430768,  0.00975126, ...,  0.00523633,
         -0.04047171, -0.0285727 ],
        [ 0.01109227,  0.03107528,  0.03501086, ...,  0.01149857,
          0.04023383,  0.03131195]], dtype=float32)>,
 <tf.Variable 'lstm/lstm_cell/recurrent_kernel:0' shape=(512, 2048) dtype=float32, numpy=
 array([[ 0.19703622,  1.8791293 , -0.6187147 , ...,  0.417162  ,
          0.28270516,  0.1432458 ],
        [-0.90168023,  1.7376618 ,  0.7338706 , ...,  1.5522743 ,
         -0.0029869

In [25]:
for l in model2.layers:
    if l.name == "lstm":
        l.trainable = False

In [27]:
epochs = 25

history = model2.fit(
    x = inputs,
    y = {"pitch": note_outputs, "duration": duration_outputs, "offset": offset_outputs},
    epochs = epochs,
    callbacks = tf.keras.callbacks.EarlyStopping(
        monitor = 'loss',
        patience = 5,
        verbose = 1,
        restore_best_weights = True),
    validation_split = 0.2
)

Epoch 1/25
61/61 [==============================] - 9s 148ms/step - loss: 32.0334 - duration_loss: 6.1610 - offset_loss: 8.4547 - pitch_loss: 17.4177 - duration_accuracy: 0.2168 - offset_accuracy: 0.1301 - pitch_accuracy: 0.0965 - val_loss: 21.1388 - val_duration_loss: 4.2612 - val_offset_loss: 6.4193 - val_pitch_loss: 10.4583 - val_duration_accuracy: 0.2268 - val_offset_accuracy: 0.1918 - val_pitch_accuracy: 0.0887
Epoch 2/25
61/61 [==============================] - 9s 143ms/step - loss: 21.1296 - duration_loss: 4.0105 - offset_loss: 6.3481 - pitch_loss: 10.7710 - duration_accuracy: 0.2354 - offset_accuracy: 0.1626 - pitch_accuracy: 0.1136 - val_loss: 16.9707 - val_duration_loss: 3.7224 - val_offset_loss: 5.8202 - val_pitch_loss: 7.4281 - val_duration_accuracy: 0.2433 - val_offset_accuracy: 0.1835 - val_pitch_accuracy: 0.1299
Epoch 3/25
61/61 [==============================] - 9s 144ms/step - loss: 17.0492 - duration_loss: 3.4768 - offset_loss: 5.6581 - pitch_loss: 7.9143 - duration_a

In [28]:
num_notes = 100
seed_notes = inputs[0]
chord_builder = []
note_stream = []
offset = 0

for g in reversed(seed_notes):
    n = note.Note(np.argmax(g[:NUM_PIANO_KEYS]) + A0_MIDI_OFFSET)
    str_duration = reverse_duration[np.argmax(g[NUM_PIANO_KEYS : NUM_PIANO_KEYS + len(duration_map)])]
    try:
        n.quarterLength = float(str_duration)
    except:
        n.quarterLength = Fraction(str_duration)
    offset = reverse_offset[np.argmax(g[-1 * len(offset_map):])]
    if offset == 0:
        chord_builder.append(n)
    elif len(chord_builder) == 0:
        note_stream.append((n, offset))
    else:
        note_stream.append((chord.Chord(chord_builder), offset))
        chord_builder = []
if len(chord_builder) == 1:
    note_stream.append((chord_builder[0], 0))
elif len(chord_builder) > 1:
    note_stream.append((chord.Chord(chord_builder), 0))
note_stream.reverse()

In [29]:
seed_stream = stream.Stream()
previous_offset = 0
for n, off in note_stream:
    previous_offset += off
    seed_stream.insert(previous_offset, n)

In [30]:
generated_notes = []
for i in range(num_notes):
    new_note = model2.predict(tf.expand_dims(seed_notes, 0))
    new_input = np.concatenate([new_note["pitch"], new_note["duration"], new_note["offset"]], axis = 1)
    generated_notes.append(new_note)
    seed_notes = np.delete(seed_notes, 0, axis = 0)
    seed_notes = np.append(seed_notes, new_input, axis = 0)

1/1 [==============================] - 0s 15ms/step


In [31]:
chord_builder = []
note_stream = []
offset = 0
for i in reversed(range(len(generated_notes))):
    g = generated_notes[i]
    n = note.Note(np.argmax(g["pitch"]) + A0_MIDI_OFFSET)
    try:
        n.quarterLength = float(reverse_duration[np.argmax(g["duration"])])
    except:
        n.quarterLength = Fraction(reverse_duration[np.argmax(g["duration"])])
    offset = reverse_offset[np.argmax(g["offset"])]
    if offset == 0:
        chord_builder.append(n)
    elif len(chord_builder) == 0:
        note_stream.append((n, offset))
    else:
        note_stream.append((chord.Chord(chord_builder), offset))
        chord_builder = []
if len(chord_builder) == 1:
    note_stream.append((chord_builder[0], 0))
elif len(chord_builder) > 1:
    note_stream.append((chord.Chord(chord_builder), 0))
note_stream.reverse()

In [32]:
s = stream.Stream()
previous_offset = 0
for n, off in note_stream:
    previous_offset += off
    s.insert(previous_offset, n)

In [33]:
seed_stream.write("midi", "seed.mid")

'seed.mid'

In [34]:
seed_stream.show("midi")

In [35]:
s.write("midi", "output.mid")

'output.mid'

In [36]:
s.show("midi")